In [42]:
#include json and os librareries
import json
import time
import pandas as pd
import os
from subprocess import run

In [43]:
####Globals:
JSON_DIR="ExtractTestsResults/JsonTestsFolders/oslats/oslat-"
TEST_TYPE = "oslat"

############################################################################
#                                                                          #
#                    #################################                     #
#                    #      Helper functions         #                     #
#                    #################################                     #
#                                                                          #
############################################################################

def create_json(results_dict:dict, JSON_DIR:str):
    #create json object
    #add uniuqe id to file name (using unix time)
    timestamp = str(time.time()).replace('.', '')
    # open a file and write the JSON data to it
    with open(JSON_DIR+timestamp+".json", "w") as json_file:
        json.dump(results_dict, json_file)

def list_files(path):
    """
    Return a list of all json file names in the specified folder.
    """
    files = []
    for file in os.listdir(path):
        if os.path.isfile(os.path.join(path, file)):
            files.append(file)
    return files

def extract_test_units(results_dict:dict, test_units_df):
    """
    extract all test units metrics(thread/cores) of a specific test, and update them in the result dict.\n
    params:
        -results_dict(dict)-dict that contains test parameters and metrics.\n
        -test_units_df(dataframe)- dataframe that contains specific test's test units metrics.\n
    returns: updated dict that contains test parameters and metrics.\n.
    """
    test_units_list = []
    for _, row in test_units_df.iterrows():
        test_units_list.append({"index":row["Index"], 
            "max_latency":row["Max"], 
            "min_latency":row["Min"], 
            "avg_latency":row["Avg"], 
            "availability":row["Availability"], 
            "number_of_nines":row["Number of Nines"]})

    results_dict["test_units"] = test_units_list

    return results_dict   


In [50]:
availabilty_df = pd.read_csv("spreadsheets/RAN QE Performance KPIs - oslat.csv").fillna("")
availabilty_df["Version"] = availabilty_df["Version"].astype("str")
availabilty_df["Operator Version"] = availabilty_df["Operator Version"].astype("str")
availabilty_df.loc[availabilty_df["Version"] == "4.1", "Version"] = "4.10"
availabilty_df.loc[availabilty_df["Operator Version"] == "4.1", "Operator Version"] = "4.10"
if not set(["Kernel", "Sideloaded"]).issubset(availabilty_df.columns):
    availabilty_df["Kernel"] = ""
    availabilty_df["Sideloaded"] = ""

In [52]:
grouped_availabilty = availabilty_df.groupby(["Version", "OCP Version"])

In [53]:
for version_key, threads in grouped_availabilty:
    print("creating json file for {}".format(version_key))
    threads = threads.sort_values(by='Index', ascending=True)
    results_dict = {}
    results_dict["ocp_version"] = threads["Version"].values[0]
    results_dict["ocp_build"] = threads["OCP Version"].values[0]
    results_dict["test_type"] = TEST_TYPE
    results_dict["node_name"] = threads["Cluster"].values[0]
    results_dict["duration"] = threads["Duration"].values[0]
    results_dict["kernel"] = threads["Kernel"].values[0]
    results_dict["sideloaded"] = "true" if threads["Sideloaded"].values[0] == "Yes" else "false" if threads["Sideloaded"].values[0] == "No" else "" 
    results_dict["operator_version"] = threads["Operator Version"].values[0]

    results_dict = extract_test_units(results_dict=results_dict, test_units_df=threads)
    create_json(results_dict=results_dict, JSON_DIR=JSON_DIR)

creating json file for ('4.10', '4.10.0-rc.0')
creating json file for ('4.10', '4.10.23')
creating json file for ('4.10', '4.10.8')
creating json file for ('4.11', '4.11.0-rc.1')
creating json file for ('4.11', '4.11.22')
creating json file for ('4.8', '4.8.15')
creating json file for ('4.9', '4.9.0 rc.0')
creating json file for ('4.9', '4.9.13-assembly.art3657')
creating json file for ('4.9', '4.9.6')


In [54]:
json_files = list_files("ExtractTestsResults/JsonTestsFolders/oslats")

for file in json_files:
    json_file_file = "ExtractTestsResults/JsonTestsFolders/oslats/" + file
    cmd = ['python3', 'splunkHecPush.py',  '-j', json_file_file, '-i', 'ecosystem-qe-dev', '-t' ,'94eeb2f9-8e5f-4c86-adec-336b3368c74e']
    print("pushing {} to splunk".format(file))
    run(cmd)

pushing oslat-1678787168340665.json to splunk


/home/alugasi/tasks/google data studio/code/data/lib/python3.10/site-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'splunk-hec.prod.utility-us-east-2.redhat.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


(200, '{"text":"Success","code":0}', {'Date': 'Tue, 14 Mar 2023 10:07:14 GMT', 'Content-Type': 'application/json; charset=UTF-8', 'Content-Length': '27', 'Connection': 'keep-alive', 'Set-Cookie': 'AWSALB=gPi0YV4xNuMh7FoAvn19Om1Na5AXUDYeoL5rNu3Cq0FqcnFISWkEd3qdBr07UbkhWqMC9eb/xQtpedmCFEYtpMtTaXetU23MX3XDVX+aU1egfoQVcL7p0/yh6EAt; Expires=Tue, 21 Mar 2023 10:07:14 GMT; Path=/, AWSALBCORS=gPi0YV4xNuMh7FoAvn19Om1Na5AXUDYeoL5rNu3Cq0FqcnFISWkEd3qdBr07UbkhWqMC9eb/xQtpedmCFEYtpMtTaXetU23MX3XDVX+aU1egfoQVcL7p0/yh6EAt; Expires=Tue, 21 Mar 2023 10:07:14 GMT; Path=/; SameSite=None; Secure', 'X-Content-Type-Options': 'nosniff', 'Vary': 'Authorization', 'X-Frame-Options': 'SAMEORIGIN', 'Server': 'Splunkd'})
pushing oslat-1678787168342307.json to splunk


/home/alugasi/tasks/google data studio/code/data/lib/python3.10/site-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'splunk-hec.prod.utility-us-east-2.redhat.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


(200, '{"text":"Success","code":0}', {'Date': 'Tue, 14 Mar 2023 10:07:16 GMT', 'Content-Type': 'application/json; charset=UTF-8', 'Content-Length': '27', 'Connection': 'keep-alive', 'Set-Cookie': 'AWSALB=dBZwHVB4xNDDrfrQhSWBhJaff6r4Ta6ExAGOZn2+/1omo33VljjZ2zb2IPfJTH7+IwibBmvTM7FCKzDZQa7BHOq+pmst7I5CWqwl2pmJ2qSDnLsQpFxqQZEcZgW6; Expires=Tue, 21 Mar 2023 10:07:16 GMT; Path=/, AWSALBCORS=dBZwHVB4xNDDrfrQhSWBhJaff6r4Ta6ExAGOZn2+/1omo33VljjZ2zb2IPfJTH7+IwibBmvTM7FCKzDZQa7BHOq+pmst7I5CWqwl2pmJ2qSDnLsQpFxqQZEcZgW6; Expires=Tue, 21 Mar 2023 10:07:16 GMT; Path=/; SameSite=None; Secure', 'X-Content-Type-Options': 'nosniff', 'Vary': 'Authorization', 'X-Frame-Options': 'SAMEORIGIN', 'Server': 'Splunkd'})
pushing oslat-16787871683435643.json to splunk


...

(200, '{"text":"Success","code":0}', {'Date': 'Tue, 14 Mar 2023 10:07:32 GMT', 'Content-Type': 'application/json; charset=UTF-8', 'Content-Length': '27', 'Connection': 'keep-alive', 'Set-Cookie': 'AWSALB=mAV2/jfs9frCESpjLqlS85P2Y885Rz7BtOIhxEOehVdj2hkZTlVRpmj2LD1UTpm8O9wfBV023O2iWU02N+ANpAXsUFU7abUmlwJC6USkS/3RU/jlEuZYubqBnCgD; Expires=Tue, 21 Mar 2023 10:07:32 GMT; Path=/, AWSALBCORS=mAV2/jfs9frCESpjLqlS85P2Y885Rz7BtOIhxEOehVdj2hkZTlVRpmj2LD1UTpm8O9wfBV023O2iWU02N+ANpAXsUFU7abUmlwJC6USkS/3RU/jlEuZYubqBnCgD; Expires=Tue, 21 Mar 2023 10:07:32 GMT; Path=/; SameSite=None; Secure', 'X-Content-Type-Options': 'nosniff', 'Vary': 'Authorization', 'X-Frame-Options': 'SAMEORIGIN', 'Server': 'Splunkd'})
pushing oslat-1678788198724117.json to splunk


/home/alugasi/tasks/google data studio/code/data/lib/python3.10/site-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'splunk-hec.prod.utility-us-east-2.redhat.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


(200, '{"text":"Success","code":0}', {'Date': 'Tue, 14 Mar 2023 10:07:33 GMT', 'Content-Type': 'application/json; charset=UTF-8', 'Content-Length': '27', 'Connection': 'keep-alive', 'Set-Cookie': 'AWSALB=QMHENip8UI58xjDqiOk5xGL8PlgUr6e8EMkbDCC+vNCovtE9PEXb/sgbUrfHFohwopULhEaUKa1K5Kn+iiyGqkcltEDzDvP3PeVMdtcSACrJeCZxuPvCClpa8Bn+; Expires=Tue, 21 Mar 2023 10:07:33 GMT; Path=/, AWSALBCORS=QMHENip8UI58xjDqiOk5xGL8PlgUr6e8EMkbDCC+vNCovtE9PEXb/sgbUrfHFohwopULhEaUKa1K5Kn+iiyGqkcltEDzDvP3PeVMdtcSACrJeCZxuPvCClpa8Bn+; Expires=Tue, 21 Mar 2023 10:07:33 GMT; Path=/; SameSite=None; Secure', 'X-Content-Type-Options': 'nosniff', 'Vary': 'Authorization', 'X-Frame-Options': 'SAMEORIGIN', 'Server': 'Splunkd'})
pushing oslat-16787881987255325.json to splunk


/home/alugasi/tasks/google data studio/code/data/lib/python3.10/site-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'splunk-hec.prod.utility-us-east-2.redhat.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


(200, '{"text":"Success","code":0}', {'Date': 'Tue, 14 Mar 2023 10:07:34 GMT', 'Content-Type': 'application/json; charset=UTF-8', 'Content-Length': '27', 'Connection': 'keep-alive', 'Set-Cookie': 'AWSALB=f1JpTicY+9oIBbMRAROcCnXLVJPXIxGDp0GOqpHmdDYHpjbeiOv+iKYdUFd/TQRyb64wAzAKbpcaNfJoT+yg8hJcJ98xx0tX6SUIKL+dp2YPGHSfHMpiJIFg8NQH; Expires=Tue, 21 Mar 2023 10:07:34 GMT; Path=/, AWSALBCORS=f1JpTicY+9oIBbMRAROcCnXLVJPXIxGDp0GOqpHmdDYHpjbeiOv+iKYdUFd/TQRyb64wAzAKbpcaNfJoT+yg8hJcJ98xx0tX6SUIKL+dp2YPGHSfHMpiJIFg8NQH; Expires=Tue, 21 Mar 2023 10:07:34 GMT; Path=/; SameSite=None; Secure', 'X-Content-Type-Options': 'nosniff', 'Vary': 'Authorization', 'X-Frame-Options': 'SAMEORIGIN', 'Server': 'Splunkd'})
